# Импортируем библиотеки `"pandas"` (библиотека для обработки и анализа данных), `"matmlotlib"` (низкоуровневая библиотека для построения графиков на `Python`, которая служит в качестве утилиты визуализации), а именно модуль `.pyplot` (модуль в пакете Matplotlib) и и `Seaborn` — это программный пакет визуализации данных для языка программирования Python, основанный на библиотеке Matplotlib.
---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
# Загрузка csv-файла в объект DataFrame.
# Метод pd.read_csv() считывает данные из файла csv (формата .csv) и загружает в таблицу.
df_walmart = pd.read_csv('./data/clean/walmart_sls.clean.csv')

In [ ]:
# Убираем научную (scientific) нотацию в выводе чисел в pandas глобально
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Удалить лишние пробелы в названиях столбцов.
df_walmart.columns = df_walmart.columns.str.strip()

# Вывести названия столбцов DataFrame.
print("Названия столбцов DataFrame:")
df_walmart.columns

In [ ]:
# Отображаем первые 5 строк датафрейма
df_walmart.head()

In [ ]:
# Отображаем последние 5 строк датафрейма
df_walmart.tail()

In [ ]:
# Функция .info() показывает информацию о DataFrame
# Включая типы данных и использование памяти
# memory_usage = "deep" позволяет получить более точную информацию о потреблении памяти
# Важно отметить, что использование memory_usage = "deep" может занять больше времени, так как оно требует дополнительного анализа
df_walmart.info(memory_usage='deep')

In [ ]:
# Просматриваем соразмерность столбца (строки и столбцы / читать слева на право)
df_walmart.shape

In [ ]:
# Метод .describe() используется для получения сводной статистики по DataFrame.
# Он автоматически рассчитывает количество ненулевых значений (count), среднее значение (mean),
# стандартное отклонение (std / разброс данных), минимальное значение (min), 25-й, 50-й (медиана) и 75-й процентили (quartiles), а также максимальное значение (max)
# для каждого числового столбца в DataFrame df_walmart.
df_walmart.describe()

Перевести колонку `Date` с типом данных object на формат `"datetime"`.
---
### Результат анализа колоноки с типом данных "время" по фактическому содержимому:
| Название колонки | Ожидаемый результат |
| ----------- | ----------- |
| `Date` | ✅ Содержит дату. Преобразовать к `datetime`. |

In [ ]:
# Преобразование столбца 'date' в формат datetime
df_walmart['date'] = pd.to_datetime(
    df_walmart['date'],
    errors='coerce',      # если значение не удаётся преобразовать, то → NaT
    dayfirst=False        # Указываем dayfirst=False, чтобы Pandas правильно понял день и месяц
)

print(df_walmart['date'].dtype)

---

## Проведите исследовательский анализ данных 
### Определите, как меняется объем продаж в зависимости от праздников и времени года.
### 1.	Постройте визуализацию, демонстрирующую динамику понедельных продаж по сети за год.
### 2.	Ответьте на вопрос: какие сезоны или недели имеют наибольшие объемы продаж?
---

### Создание нового признака `season` (времена года)
### Для анализа сезонности вводим категориальный признак на основе месяца
### Функция `get_season()` присваивает каждому месяцу соответствующий сезон:
### Winter (декабрь–февраль), Spring (март–май), Summer (июнь–август), Autumn (сентябрь–ноябрь)
### Признак используется для группировки продаж и выявления сезонных закономерностей
---

In [ ]:
# Функция для получения сезона по месяцу
# Параметр month должен быть целым числом от 1 до 12
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'
    
# Применяем функцию get_season к столбцу 'date' и создаём новый столбец 'season'
df_walmart['season'] = df_walmart['date'].dt.month.apply(get_season)

# Проверяем, что новый столбец 'season' был успешно добавлен (общее распределение данных по сезонам)
df_walmart['season'].value_counts()

### Визуализация общей выручки по сезонам
### Группируем продажи по признаку `season` и вычисляем суммарные продажи за каждый сезон
### Столбчатая диаграмма позволяет сравнить общий вклад каждого сезона в общую выручку сети Walmart
### Это помогает определить, в какие сезоны бизнес получает наибольшую прибыль в совокупности
---

In [ ]:
# Вычисляем общие продажи по сезонам
total_sales_by_season = (
    df_walmart.groupby('season')['weekly_sales']
    .sum()
    .reset_index()
    .rename(columns={'weekly_sales': 'total_sales'})
)

# Построение графика общих продаж по сезонам
plt.figure(figsize=(10, 6))

# Создание столбчатой диаграммы
plt.bar(
    total_sales_by_season['season'],
    total_sales_by_season['total_sales'],
    color=['#FF8C00', '#3CB371', '#FFD700', '#4682B4'],
    edgecolor='black',  # Добавляем черную окантовку для лучшей видимости
)

# Настройка внешнего вида графика
plt.title('Общие продажи по сезонам', fontsize=16)
plt.xlabel('Сезоны', fontsize=14)
plt.ylabel('Общие продажи', fontsize=14)
plt.legend(['Общие продажи'])
plt.xticks(rotation=0)
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Добавляем сетку по оси Y
plt.tight_layout()  # Автоматическая настройка отступов
plt.show()

### Анализ средней недельной выручки по сезонам
### Группируем данные по признаку `season` и вычисляем среднее значение продаж за неделю
### Столбчатая диаграмма позволяет сравнить интенсивность продаж в разных сезонах
### Это помогает выявить, когда продажи наиболее активны по средней нагрузке на неделю
---

In [ ]:
# Вычисляем средние продажи по сезонам
avg_sales_by_season = (
    df_walmart.groupby('season')['weekly_sales']
    .mean()
    .reset_index()
    .rename(columns={'weekly_sales': 'avg_sales'})
)

# Построение графика средних продаж по сезонам
plt.figure(figsize=(10, 6))

# Создание столбчатой диаграммы
plt.bar(
    avg_sales_by_season['season'],
    avg_sales_by_season['avg_sales'],
    color=['#FF8C00', '#3CB371', '#FFD700', '#4682B4'],
    edgecolor='black'  # Добавляем черную окантовку для лучшей видимости
)

# Настройка внешнего вида графика
plt.title('Средние продажи по сезонам')
plt.xlabel('Сезоны')
plt.ylabel('Средние продажи')
plt.legend(['Средние продажи'])
plt.xticks(rotation=0)
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Добавляем сетку по оси Y
plt.tight_layout()  # Автоматическая настройка отступов
plt.show()

### Анализ влияния праздничных недель на среднюю выручку
### Группируем данные по признаку `holiday_flag` (1 — праздничная неделя, 0 — обычная)
### Вычисляем среднее значение `weekly_sales` для каждой группы
### Столбчатая диаграмма показывает разницу в средней выручке между праздничными и обычными неделями
---

In [ ]:
# Вычисляем средние продажи по пораздникам
avg_sales_by_holiday = (
    df_walmart.groupby('holiday_flag')['weekly_sales']
    .mean()
    .reset_index()
    .rename(columns={'weekly_sales': 'avg_sales_holiday'})
)

# Построение графика средних продаж по сезонам
plt.figure(figsize=(10, 6))

# Создание столбчатой диаграммы
plt.bar(
    avg_sales_by_holiday['holiday_flag'],
    avg_sales_by_holiday['avg_sales_holiday'],
    color=['#4682B4', 'red'],
    edgecolor='black'  # Добавляем черную окантовку для лучшей видимости
)

# Настройка внешнего вида графика
plt.title('Средние продажи по праздникам', fontsize=16)
plt.xlabel('Праздники', fontsize=14)
plt.ylabel('Средние продажи', fontsize=14)
plt.legend(['Средние продажи'])
plt.xticks(rotation=0)
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Добавляем сетку по оси Y
plt.tight_layout()  # Автоматическая настройка отступов
plt.show()

### Построение линейного графика понедельных продаж по дате
### Группируем продажи по каждой дате (неделе) и отображаем динамику во времени
### График помогает визуально определить тренды, пики и спады спроса с 2010 по 2012 год
---

In [ ]:
# Группируем данные по дате (неделя) и суммируем продажи по всей сети
weekly_sales_by_date = (
    df_walmart.groupby('date')['weekly_sales']
    .sum()
    .reset_index()
    .sort_values(by='date', ascending=True) 
)

# Построение линейного графика еженедельных продаж
plt.figure(figsize=(14, 6))

# Создание линейного графика
plt.plot(
    weekly_sales_by_date['date'],
    weekly_sales_by_date['weekly_sales'],
    color='royalblue',
    marker='o',
    linewidth=2
)

# Настройка внешнего вида графика
plt.title('Динамика понедельных продаж по сети Walmart (2010 - 2012)', fontsize=16)
plt.xlabel('Дата (неделя)', fontsize=14)
plt.ylabel('Суммарные продажи, $ ', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7) # Добавление сетки
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

---
### Извлекаем номер недели из даты и считаем средние продажи по неделям года
### Это позволит определить, какие недели приносят наибольшую выручку вне зависимости от года
---

In [ ]:
# Добавляем столбец "week" с номером недели в году
# Используем метод isocalendar() для получения номера недели в году
df_walmart['week'] = df_walmart['date'].dt.isocalendar().week

---
### Строим линейный график средних продаж по неделям календарного года `(1–52)`
### Визуализация позволяет выявить пики и провалы спроса по неделям
---

In [ ]:
# Группируем данные по неделям и вычисляем средние продажи
avg_sales_by_week = (
    df_walmart.groupby('week')['weekly_sales']
    .mean()
    .reset_index()
    .rename(columns={'weekly_sales': 'avg_weekly_sales'})
)

# Построение линейного графика средних продаж по неделям
plt.figure(figsize=(12, 5))

# Создание линейного графика
plt.plot(
    avg_sales_by_week['week'],
    avg_sales_by_week['avg_weekly_sales'],
    color='teal',
    marker='o',
    linewidth=2
)

# Настройка внешнего вида графика
plt.title('Средние продажи по неделям года (1 - 52)', fontsize=16)
plt.xlabel('Номер недели в году', fontsize=14)
plt.ylabel('Средняя недельная выручка, $', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7) # Добавление сетки
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

### Выводим топ-5 недель года с наибольшей средней выручкой
### Эти недели являются стратегически важными для бизнеса
---

In [ ]:
top_weeks = (
    avg_sales_by_week
    .sort_values(by='avg_weekly_sales', ascending=False)
    .head(5)
)

top_weeks

---
### Исследуйте влияние макроэкономических факторов `(CPI, Unemployment)` на продажи.
## 1.	Постройте график для анализа связи между данными о продажах и этими показателями.
## 2.	Ответьте на вопрос: какие факторы влияют на продажи сильнее?
---

### Построение графика рассеяния влияния между `CPI` и средними продажами по неделям `(продажами)`
---

In [ ]:
# Построение графика рассеяния влияния между CPI и средними продажами по неделям (продажами)
plt.figure(figsize=(10, 5))

# Создание графика рассеяния и линии тренда (regression line)
sns.regplot(
    data=df_walmart,
    x='cpi',
    y='weekly_sales',
    scatter_kws={'alpha': 0.2}, # Настройка внешнего вида точек
    line_kws={'color': 'red', 'linewidth': 2} # Настройка внешнего вида линии тренда
)

# Настройка внешнего вида графика
plt.title('Зависимость продаж от потребительского индекса (CPI)', fontsize=16)
plt.xlabel('Потребительский индекс (CPI)', fontsize=14)
plt.ylabel ('Недельные продажи, $', fontsize=14)
plt.ticklabel_format(style='plain', axis='y') # убираем научную нотацию на оси Y
plt.grid(True, linestyle='--', alpha=0.7) # Добавление сетки
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

### Построение графика рассеяния влияния между `Unemployment` и средними продажами по неделям `(продажами)`
---

In [ ]:
# Построение графика рассеяния влияния между Unemployment и средними продажами по неделям (продажами)
plt.figure(figsize=(10, 5))

# Создание графика рассеяния и линии тренда (regression line)
sns.regplot(
    data=df_walmart,
    x='unemployment',
    y='weekly_sales',
    scatter_kws={'alpha': 0.2}, # Настройка внешнего вида точек
    line_kws={'color': 'red', 'linewidth': 2} # Настройка внешнего вида линии тренда
)

# Настройка внешнего вида графика
plt.title('Зависимость продаж от уровня безработицы', fontsize=16)
plt.xlabel('Уровень безработицы (%)', fontsize=14)
plt.ylabel('Недельные продажи, $', fontsize=14)
plt.ticklabel_format(style='plain', axis='y') # убераем научную нотацию на оси Y
plt.grid(True, linestyle='--', alpha=0.7) # добавляем сетку
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

### Вычислим коэффициенты корреляций Пирсона `(Pearson correlation coefficient)`.
---

In [ ]:
# Метод .corr() вычисляет коэффициент корреляции между двумя столбцами DataFrame
# Корреляция показывает, насколько сильно связаны два признака 
# (в данном случае CPI и weekly_sales, Unemployment и weekly_sales)

# Вычисляем корреляцию между CPI и средними продажами
corr_cpi = df_walmart['cpi'].corr(df_walmart['weekly_sales'])

# Вычисляем корреляцию между уровнем безработицы и средними продажами
corr_unemployment = df_walmart['unemployment'].corr(df_walmart['weekly_sales'])

# {corr_cpi:.2f} - форматирует число с двумя знаками после запятой. Аналогично для corr_unemployment
# Выводим результаты корреляций
print(f'Коэффициент корреляции между CPI и средними продажами: {corr_cpi:.2f}')
print(f'Коэффициент кореляции между уровнем безработицы и средними продажами: {corr_unemployment:.2f}')

---
## Оцените зависимость продаж от `погодных условий` и изменения цен на `топливо` в регионе.
### 1. Создайте визуализацию, отражающую эту зависимость.
### 2. Ответьте на вопрос: как изменения погодных условий и стоимости топлива могут повлиять на продажи?
### 3. На основе анализа предложите рекомендации: как компании следует адаптировать стратегию продаж, учитывая внешние условия?
---

### Построение графика рассеяния влияния между температурой и продажами по неделям (продажами)
---

In [ ]:
# Построение графика рассеяния влияния между температурой и продажами по неделям (продажами)
plt.figure(figsize=(10, 5))

# Создание графика рассеяния и линии тренда (regression line)
sns.regplot(
    data=df_walmart,
    x='temperature',
    y='weekly_sales',
    scatter_kws={'alpha': 0.2, 'color': 'purple'}, # Настройка внешнего вида точек
    line_kws={'color': 'red', 'linewidth': 2} # Настройка внешнего вида линии тренда
)

# Настройка внешнего вида графика
plt.title('Зависимость продаж от погодных условий', fontsize=16)
plt.xlabel('Температура (F)', fontsize=14)
plt.ylabel('Недельные продажи, $', fontsize=14)
plt.ticklabel_format(style='plain', axis='y') # убераем научную нотацию на оси Y
plt.grid(True, linestyle='--', alpha=0.7) # добавляем сетку
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

### Построение графика рассеяния влияния между ценой топлива и продажами по неделям (продажами)
---

In [ ]:
# Построение графика рассеяния влияния между ценой топлива и продажами по неделям (продажами)
plt.figure(figsize=(10, 5))

# Создание графика рассеяния и линии тренда (regression line)
sns.regplot(
    data=df_walmart,
    x='fuel_price',
    y='weekly_sales',
    scatter_kws={'alpha': 0.2, 'color': 'purple'}, # Настройка внешнего вида точек
    line_kws={'color': 'red', 'linewidth': 2} # Настройка внешнего вида линии тренда
)

# Настройка внешнего вида графика
plt.title('Зависимость продаж от цены топлива', fontsize=16)
plt.xlabel('Цена топлива', fontsize=14)
plt.ylabel('Недельные продажи, $', fontsize=14)
plt.ticklabel_format(style='plain', axis='y') # убераем научную нотацию на оси Y
plt.grid(True, linestyle='--', alpha=0.7) # добавляем сетку
plt.tight_layout() # Автоматическая настройка отступов
plt.show()

### Вычислим коэффициенты корреляций Пирсона `(Pearson correlation coefficient)`.
---

In [ ]:
# Метод .corr() вычисляет коэффициент корреляции между двумя столбцами DataFrame
# Корреляция показывает, насколько сильно связаны два признака 
# (в данном случае temperature и weekly_sales, fuel_price и weekly_sales)

# Вычисляем корреляцию между температурой и средними продажами
corr_temperature = df_walmart['temperature'].corr(df_walmart['weekly_sales'])

# Вычисляем корреляцию между ценой топлива и продажами
corr_fuel_price = df_walmart['fuel_price'].corr(df_walmart['weekly_sales'])

# {corr_temperature:.2f} - форматирует число с двумя знаками после запятой. Аналогично для corr_fuel_price
# Выводим результаты корреляций
print(f'Коэффициент корреляции между temperature и продажами: {corr_temperature:.2f}')
print(f'Коэффициент корреляции между fuel_price и продажами: {corr_fuel_price:.2f}')